In [1]:
import os, sys

sys.path.append(os.path.abspath("Datasets"))
sys.path.append(os.path.abspath("Images"))

### Multiple Linear Regression model:
$$ y = b_0 + b_1*x_1 + b_2*x_2 + ... + b_n*x_n$$

<img src='Images/3_1_assumptions.JPG'></img>

- Before building the model, we need to check that those assumptions are true
- **linear**: linear regression needs the *relationship* between the independent and dependent variables to be linear (x, y). Outliers should also be checked since linear regression is sensitive to outlier effects.
- **Homoscedasticity**: error terms along the regression are equal (variance is unchanged). If the data is heteroscedastic, the scatter plot looks like the following example:
<img src='Images/3_2_homoscedasticity.JPG'><img src='Images/3_2_homoscedasticity_2.JPG'>
- **Multivariate normality**: all variables to be multivariate normal. All variables' distribution are Gaussian
- **Independence of errors**: no correlation between consecutive errors in the case of time series data
- **lack of multicollinearity**: two or more independent variables are not correlated, it means not any two of them is a linear combination of the other.

### Dummy variables
- Should only include K-1 column for K values of categorical data  (to avoid dummy variable trap)
<img src="Images/3_3_dummy.JPG">
- As the image above, if $D_1=0$, it means the "California"'s coefficient is included in $b_0$ => $b_4$ is the difference between "New York" and "California"

### Dummy Variable Trap
- We should not include both at the same time:
<img src="Images/3_3_dummy_trap.JPG">
- We have $D_2 = 1 - D_1$ always, so we will always have collinearity. (California's column is linear dependent with New York's column)

$=>$ **Always omit one dummy variable**

## Step-by-step building a model
- There are too many independent variables, so we need to choose which one to keep, which one to throw out
<img src='Images/3_4_model_1.JPG'>
- Reasons:
    - If we throw in a lot of variables, it's not a reliable model anymore
    - Too many variables to be explained, you mess up yourself
#### Methods of building models:
<img src='Images/3_4_model_2.JPG'>
- Stepwise regression: 2 or 3 or 4, sometimes for just 4

##### All-in: All variables have to be included
- Having prior knowledge; OR
- Have to; OR
- Preparing for Backward Elimination

##### Backward Elimination: 
<img src='Images/3_4_backward_elimination_2.JPG'>
<img src='Images/3_4_backward_elimination_1.JPG'>
- After step 5, goes back to step 3. If P < SL, stop, the model is ready
- P <SL: the variable is not significant anymore

##### Forward Selection:
<img src='Images/3_5_forward_selection_1.JPG'>
<img src='Images/3_5_forward_selection_2.JPG'>

##### Bidirectional Elimination:
<img src='Images/3_6_bidirectional.JPG'>
<img src='Images/3_6_bidirectional_2.JPG'>

##### All Possbile Models:
- Resources consume
<img src='Images/3_7_all.JPG'>

$=>$ Should use backward elimination

### Coding Part

In [2]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

In [3]:
dataset = pd.read_csv("Datasets/ML_a_z/50_Startups.csv")

In [4]:
dataset.head()

,R&D Spend,Administration,Marketing Spend,State,Profit
0,165349.20,136897.80,471784.10,New York,192261.83
1,162597.70,151377.59,443898.53,California,191792.06
2,153441.51,101145.55,407934.54,Florida,191050.39
3,144372.41,118671.85,383199.62,New York,182901.99
4,142107.34,91391.77,366168.42,Florida,166187.94


- Profit is the dependent variable
- Others are independent variables

In [5]:
X = dataset.iloc[:, :-1].values
Y = dataset.iloc[:, -1].values

In [6]:
# Deal with categorical data
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

label_encoder_x = LabelEncoder()

In [7]:
X[:, -1] = label_encoder_x.fit_transform(X[:, -1])
# Label encoder: translate to number
one_hot_encoder = OneHotEncoder(categorical_features=[-1])
# One hot encoder: translate to one hot code
X = one_hot_encoder.fit_transform(X).toarray()

In [8]:
# Avoiding the Dummy variable trap
X = X[:, 1:] # Eliminate first column from X 

In [9]:
# Splitting the dataset to training and test set
from sklearn.cross_validation import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y,
                                                   test_size=0.2,
                                                   random_state=0)

C:\Users\HuuTanVu\Anaconda2\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [10]:
X_test.shape

(10L, 5L)

In [11]:
from sklearn.linear_model import LinearRegression

regressor = LinearRegression()

In [12]:
regressor.fit(X_train, Y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [13]:
# Predicting the test results
y_pred = regressor.predict(X_test)

In [14]:
### Building the optimal model using Backward Elimination
import statsmodels.formula.api as sm

- For $b_0$ without any x dependent, we will assign an $x_0=0$ for it
$$y = b_0*x_0 + b_1*x_1 + .... +b_n*x_n$$
- We need to add a new column of all 1 into the dataset at the beginning position

In [15]:
X = np.append(np.ones((X.shape[0], 1)).astype(int), X,  axis=1)

- Need statistical information of independent variable

In [17]:
X_opt = X[:, [0, 1, 2, 3, 4, 5, ]] 
# Write this because we gonna remove after each step


- Step 1: Select a significance level to stay in model

In [18]:
significant_level = 0.05
# Or
SL = 0.05

- Step 2: Fit all possible predictors

In [19]:
# Create a new regressor from Statsmodels
# OLS: Ordinary least square
# Need to add the intercept before adding to exog
regressor_OLS = sm.OLS(endog=Y, exog = X_opt).fit()

- Step 3: Consider predictor with highest P-value. If P > SL, go to step 4, or Finish

In [21]:
regressor_OLS.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.951
Model:                            OLS   Adj. R-squared:                  0.945
Method:                 Least Squares   F-statistic:                     169.9
Date:                Wed, 01 Feb 2017   Prob (F-statistic):           1.34e-27
Time:                        23:35:43   Log-Likelihood:                -525.38
No. Observations:                  50   AIC:                             1063.
Df Residuals:                      44   BIC:                             1074.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const       5.013e+04   6884.820      7.281      0.000      3.62e+04   6.4e+04
x1           198.7888   3371.007      0.059      0.953     -6595.030  6992.607
x2           -41.8870   3256.039     -0.013      0.990     -6604.003  6520.229
x3             0.8060      0.046     17.369      0.000         0.712     0.900
x4            -0.0270      0.052     -0.517      0.608        -0.132     0.078
x5             0.0270      0.017      1.574      0.123        -0.008     0.062
==============================================================================
Omnibus:                       14.782   Durbin-Watson:                   1.283
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               21.266
Skew:                          -0.948   Prob(JB):                     2.41e-05
Kurtosis:                       5.572   Cond. No.                     1.45e+06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.45e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

- x2's P value > 0.05 => Step 4: Remove the predictor

In [22]:
X_opt = X[:, [0, 1, 3, 4, 5, ]] 
# Write this because we gonna remove after each step
regressor_OLS = sm.OLS(endog=Y, exog = X_opt).fit()

In [23]:
regressor_OLS.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.951
Model:                            OLS   Adj. R-squared:                  0.946
Method:                 Least Squares   F-statistic:                     217.2
Date:                Wed, 01 Feb 2017   Prob (F-statistic):           8.49e-29
Time:                        23:38:22   Log-Likelihood:                -525.38
No. Observations:                  50   AIC:                             1061.
Df Residuals:                      45   BIC:                             1070.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const       5.011e+04   6647.870      7.537      0.000      3.67e+04  6.35e+04
x1           220.1585   2900.536      0.076      0.940     -5621.821  6062.138
x2             0.8060      0.046     17.606      0.000         0.714     0.898
x3            -0.0270      0.052     -0.523      0.604        -0.131     0.077
x4             0.0270      0.017      1.592      0.118        -0.007     0.061
==============================================================================
Omnibus:                       14.758   Durbin-Watson:                   1.282
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               21.172
Skew:                          -0.948   Prob(JB):                     2.53e-05
Kurtosis:                       5.563   Cond. No.                     1.40e+06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.4e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [24]:
X_opt = X[:, [0, 3, 4, 5, ]] 
# Write this because we gonna remove after each step
regressor_OLS = sm.OLS(endog=Y, exog = X_opt).fit()

In [25]:
regressor_OLS.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.951
Model:                            OLS   Adj. R-squared:                  0.948
Method:                 Least Squares   F-statistic:                     296.0
Date:                Wed, 01 Feb 2017   Prob (F-statistic):           4.53e-30
Time:                        23:38:58   Log-Likelihood:                -525.39
No. Observations:                  50   AIC:                             1059.
Df Residuals:                      46   BIC:                             1066.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const       5.012e+04   6572.353      7.626      0.000      3.69e+04  6.34e+04
x1             0.8057      0.045     17.846      0.000         0.715     0.897
x2            -0.0268      0.051     -0.526      0.602        -0.130     0.076
x3             0.0272      0.016      1.655      0.105        -0.006     0.060
==============================================================================
Omnibus:                       14.838   Durbin-Watson:                   1.282
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               21.442
Skew:                          -0.949   Prob(JB):                     2.21e-05
Kurtosis:                       5.586   Cond. No.                     1.40e+06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.4e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [26]:
X_opt = X[:, [0, 3, 5 ]] 
# Write this because we gonna remove after each step
regressor_OLS = sm.OLS(endog=Y, exog = X_opt).fit()

In [27]:
regressor_OLS.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.950
Model:                            OLS   Adj. R-squared:                  0.948
Method:                 Least Squares   F-statistic:                     450.8
Date:                Wed, 01 Feb 2017   Prob (F-statistic):           2.16e-31
Time:                        23:39:31   Log-Likelihood:                -525.54
No. Observations:                  50   AIC:                             1057.
Df Residuals:                      47   BIC:                             1063.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const       4.698e+04   2689.933     17.464      0.000      4.16e+04  5.24e+04
x1             0.7966      0.041     19.266      0.000         0.713     0.880
x2             0.0299      0.016      1.927      0.060        -0.001     0.061
==============================================================================
Omnibus:                       14.677   Durbin-Watson:                   1.257
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               21.161
Skew:                          -0.939   Prob(JB):                     2.54e-05
Kurtosis:                       5.575   Cond. No.                     5.32e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 5.32e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [28]:
X_opt = X[:, [0, 3 ]] 
# Write this because we gonna remove after each step
regressor_OLS = sm.OLS(endog=Y, exog = X_opt).fit()

In [29]:
regressor_OLS.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.947
Model:                            OLS   Adj. R-squared:                  0.945
Method:                 Least Squares   F-statistic:                     849.8
Date:                Wed, 01 Feb 2017   Prob (F-statistic):           3.50e-32
Time:                        23:42:24   Log-Likelihood:                -527.44
No. Observations:                  50   AIC:                             1059.
Df Residuals:                      48   BIC:                             1063.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const       4.903e+04   2537.897     19.320      0.000      4.39e+04  5.41e+04
x1             0.8543      0.029     29.151      0.000         0.795     0.913
==============================================================================
Omnibus:                       13.727   Durbin-Watson:                   1.116
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               18.536
Skew:                          -0.911   Prob(JB):                     9.44e-05
Kurtosis:                       5.361   Cond. No.                     1.65e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.65e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

- We see that, only 1 variable left, so the left over have the strongest affect to the model